<a href="https://colab.research.google.com/github/t60011/stable_diffusion_1_5_webui_colab/blob/main/stable_diffusion_1_5_webui_colab_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion WebUI 雲端版使用說明

*   點擊“**程式碼儲存格(一)**”左邊的 “▶︎“ 圖示開始執行。
*   第一次使用還沒有備份設定檔案先不用勾選”apply_backup_config“
*   每次開始執行需要在雲端下載很多檔案，請耐心等待儲存格下方出現**連線網址**。
*   結束使用前，先按“**程式碼儲存格(一)**”左邊的 “▶︎“ 停止執行，然後再執點擊“**程式碼儲存格(二)**”跟“**程式碼儲存格(三)**”備份設定檔跟生成圖像到自己的 Google 雲端硬碟。
*   離線之前，記得要點選右上方連線的“**中斷連線並刪除執行階段**”，否則停止連線後還是會消耗運算額度。免費額度如果耗盡，會需要等待一段時間（12-48小時），或是付費之後才能使用。
*   意外離線或關閉視窗但沒有刪除執行階段，只要在Colab打開本筆記，按右上方的“連線至代管的執行階段”就可以。



In [ ]:
# @title ＊ 程式碼儲存格(一)：Stable Diffusion 1.5 Coㄨㄩ委員翻入lab Lite 主程式＊ 
from google.colab import drive
drive.mount('/content/drive')

# @markdown 勾選以下項目，再點擊左邊“▶︎“圖示執行本儲存格。
# @markdown ---
# @markdown ＊＊ Ａ．是否在自己的 Google 雲端硬碟創建一個 SD_Data 備份資料夾？</br>
# @markdown （第一次執行請勾選！
# @markdown 若已有 SD_Data 同名資料夾則不會再創建。）
create_SD_Data_folder = True # @param {type:"boolean"}
if create_SD_Data_folder:
  !mkdir /content/drive/MyDrive/SD_Data
  !mkdir /content/drive/MyDrive/SD_Data/settings
  !mkdir /content/drive/MyDrive/SD_Data/outputs

%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers==0.0.16 triton==2.0.0 -U

!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py
!git clone -b v2.1 https://github.com/camenduru/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone -b v2.1 https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone -b v2.1 https://github.com/camenduru/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone -b v2.1 https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
!git clone -b v2.1 https://github.com/camenduru/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui-catppuccin /content/stable-diffusion-webui/extensions/stable-diffusion-webui-catppuccin
!git clone -b v2.1 https://github.com/camenduru/a1111-sd-webui-locon /content/stable-diffusion-webui/extensions/a1111-sd-webui-locon
!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui-rembg /content/stable-diffusion-webui/extensions/stable-diffusion-webui-rembg
!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui-two-shot /content/stable-diffusion-webui/extensions/stable-diffusion-webui-two-shot
!git clone -b v2.1 https://github.com/camenduru/sd_webui_stealth_pnginfo /content/stable-diffusion-webui/extensions/sd_webui_stealth_pnginfo
!git clone -b v2.1 https://github.com/camenduru/sd-webui-aspect-ratio-helper /content/stable-diffusion-webui/extensions/sd-webui-aspect-ratio-helper
!git clone -b v2.1 https://github.com/camenduru/asymmetric-tiling-sd-webui /content/stable-diffusion-webui/extensions/asymmetric-tiling-sd-webui
# 安裝中文與雙語介面翻譯
!git clone https://github.com/harukaxxxx/stable-diffusion-webui-localization-zh_TW.git /content/stable-diffusion-webui/extensions/stable-diffusion-webui-localization-zh_TW
!git clone https://github.com/journey-ad/sd-webui-bilingual-localization.git /content/stable-diffusion-webui/extensions/sd-webui-bilingual-localization

%cd /content/stable-diffusion-webui
!git reset --hard
!git -C /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai reset --hard

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o v1-5-pruned-emaonly.ckpt

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /content/stable-diffusion-webui/modules/shared.py

# @markdown ---
# @markdown ＊＊ Ｂ. Lite版只有安裝 Stable Diffusion 1.5 基本模型(Checkpoint):

# @markdown ---
# @markdown ＊＊ Ｃ. 是否套用 Google 雲端硬碟備份的設定檔？</br>
# @markdown （首次使用還沒有執行過下個程式碼儲存格(二)不用勾！）</br>
# @markdown （確定已執行過程式碼儲存格(二)且雲端硬碟有備份設定檔再勾！)
apply_backup_config = False # @param {type:"boolean"}
if apply_backup_config:
  # Sync User Settings
  !cp -av "/content/drive/MyDrive/SD_Data/settings/config.json" "/content/stable-diffusion-webui/"
  !cp -av "/content/drive/MyDrive/SD_Data/settings/ui-config.json" "/content/stable-diffusion-webui/"
  !cp -av "/content/drive/MyDrive/SD_Data/settings/cache.json" "/content/stable-diffusion-webui/"

# @markdown ---

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

In [ ]:
# @title ＊ 程式碼儲存格(二)：備份目前的 Stable Diffusion 設定檔到自己的 Google 雲端硬碟 ＊
# @markdown 執行過一次儲存格(一)並中斷後，再按“▶︎“執行本儲存格！
# @markdown ---
# @markdown ＊ 備份設定檔可以保持中文介面等設定，不用反覆重新啟動！
!cp -av "/content/stable-diffusion-webui/config.json" "/content/drive/MyDrive/SD_Data/settings/config.json"
!cp -av "/content/stable-diffusion-webui/ui-config.json" "/content/drive/MyDrive/SD_Data/settings/ui-config.json"
!cp -av "/content/stable-diffusion-webui/cache.json" "/content/drive/MyDrive/SD_Data/settings/cache.json"

In [ ]:
# @title ＊ 程式碼儲存格(三)：備份 Stable Diffusion 輸出圖像到自己的 Google 雲端硬碟 ＊
# @markdown 先按“▶︎“執行本儲存格以儲存生成圖像，再『刪除執行階段』。
# @markdown ---
# @markdown 1. 意外中斷連線請選『連線至代管的執行階段』重新連線。</br>
# @markdown 2. 『刪除執行階段』後，沒有備份的圖像都會被刪除！</br>
# @markdown 3. 要結束程式務必選『中斷所有連線並刪除執行階段』，以免運算額度耗盡！</br>
# @markdown 4. 免費額度若耗盡後，24到48小時內可能都暫時無法使用！
!cp -av -f "/content/stable-diffusion-webui/outputs/" "/content/drive/MyDrive/SD_Data/"

# 參考資料
本筆記本是以 Stable Diffusion 1.5 WebUI Colab 輕量版為基礎加以修改，原版功能包含：

*   預先載入 [runwayml/stable-diffusion-v1-5](https://huggingface.co/runwayml/stable-diffusion-v1-5) 模型，等待下載時間較短。
*   預先載入基本擴充功能。

本筆記本的**新增功能**為：
*   預先安裝**中文以及雙語介面**擴充功能，但第一次套用中文介面仍需要中斷程式並重新執行。
*   備份**設定檔案**到自己的 Google雲端硬碟，以免每次要顯示中文介面都要反覆中斷執行階段。
*   備份**生成圖像**到自己的 Google雲端硬碟，以免執行階段結束後圖像被刪除。

---
參考資料：

Stable Diffusion WebUI Colab 主頁：
https://github.com/camenduru/stable-diffusion-webui-colab 

程式碼來源：
https://colab.research.google.com/github/camenduru/stable-diffusion-webui-colab/blob/main/lite/stable_diffusion_1_5_webui_colab.ipynb

新增功能程式碼參考來源：
https://colab.research.google.com/drive/1ZxDog-yeEcXZQzcwABohSBgVhpuOO_X3